In [ ]:
#개인 신용 위험 예측 모델 제작

In [ ]:
# 1. 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


In [ ]:
# 2. 가상 데이터 생성
# 가상 데이터 만들기
np.random.seed(42)
n_samples = 1000

data = pd.DataFrame({
    'age': np.random.randint(20, 70, n_samples),
    'income': np.random.randint(2000, 10000, n_samples),  # 월 소득
    'loan_amount': np.random.randint(500, 5000, n_samples),  # 기존 대출
    'credit_cards': np.random.randint(0, 5, n_samples),  # 보유 카드 수
    'job_type': np.random.choice(['office', 'self-employed', 'unemployed'], n_samples),
    'defaulted': np.random.choice([0, 1], n_samples, p=[0.8, 0.2])  # 연체 여부
})

print(data.head())


In [ ]:
# 3. 데이터 전처리
# 수치형, 범주형 변수 구분
num_features = ['age', 'income', 'loan_amount', 'credit_cards']
cat_features = ['job_type']

# 전처리 파이프라인
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(), cat_features)
    ])

X = data.drop('defaulted', axis=1)
y = data['defaulted']

X_processed = preprocessor.fit_transform(X)

# 학습/검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)


In [ ]:
# 4. 딥러닝 모델 생성

model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # 0~1 확률 출력
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
modelpath="personal_credit.h5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
# 5. 모델 학습
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=,
                   # 콜백 리스트 추가 
                    callbacks=[early_stopping_callback, checkpointer] )


In [ ]:
# 6. 샘플 예측 (새로운 데이터 입력)
sample_data = pd.DataFrame({
    'age': [35],
    'income': [5000],
    'loan_amount': [2000],
    'credit_cards': [2],
    'job_type': ['office']
})

# 동일한 전처리 적용
sample_processed = preprocessor.transform(sample_data)

# 예측
sample_pred_prob = model.predict(sample_processed)
sample_pred = (sample_pred_prob > 0.5).astype(int)

# 클래스 라벨 매핑
label_map = {0: "연체 없음(정상)", 1: "연체 있음(위험 고객)"}

print("샘플 예측 확률:", sample_pred_prob[0][0])
print("샘플 예측 결과:", label_map[sample_pred[0][0]])

